The idea is to create a TTF comprised of the companies in the following sectors:
- Bank
- Insurance
- Energy
- Real Estate 
- Utilities
- Food & Staples Retailing

Each sector is arranged in its own sleeve by market cap. THe weight between the sleeves are done based on Sharpe optimization

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import scipy.optimize as sco
from optimizer import GainyOptimizer
from ttf_filtering import FilterTTF
from connect import GetQuery

In [2]:
dt_today = "2022-11-24"

# Select stocks from sectors

We established which sectors usually perform well during inflation

In [3]:
groups_to_select = ['Utilities', 'Banks', 'Insurance', 'Real Estate', 'Energy', 'Food & Staples Retailing']

In [4]:
tickers = GetQuery(f"""
    SELECT tm.*, tg.gic_group
    FROM
    (select symbol, gic_group from base_tickers where gic_group IN {tuple(groups_to_select)}) tg
    LEFT JOIN
    (select symbol, market_capitalization from ticker_metrics) tm
    ON tg.symbol = tm.symbol
    
""")
tickers

,symbol,market_capitalization,gic_group
0,PRT,9.598737e+07,Energy
1,PEI,1.670392e+07,Real Estate
2,OBAS,6.687279e+07,Real Estate
3,BSBK,1.595742e+08,Banks
4,PBR-A,6.292358e+10,Energy
...,...,...,...
1221,HBT,5.796524e+08,Banks
1222,ICCH,4.833674e+07,Insurance
1223,KFS,1.739327e+08,Insurance
1224,UMPQ,4.395323e+09,Banks


In [5]:
# Combine in one db
df = tickers.copy()
df = df.loc[~df.symbol.isin(['LFC','DCUE'])]

In [6]:
df.groupby('gic_group').market_capitalization.count()

gic_group
Banks                       431
Energy                      295
Food & Staples Retailing     29
Insurance                   108
Real Estate                 245
Utilities                   107
Name: market_capitalization, dtype: int64

# Create market cap weighted sleeves of stocks - max 5 stocks per sleeve

In [7]:
# Select top-10 in market cap
df = df.sort_values(['gic_group', 'market_capitalization'], ascending=False).groupby('gic_group').head(5).reset_index()
df

,index,symbol,market_capitalization,gic_group
0,1084,NEE,1.695842e+11,Utilities
1,1191,DUK,7.671367e+10,Utilities
2,134,SO,7.284310e+10,Utilities
3,391,SRE,5.138087e+10,Utilities
4,734,D,5.127141e+10,Utilities
5,1103,PLD,1.074372e+11,Real Estate
6,350,AMT,1.024705e+11,Real Estate
7,525,EQIX,6.243175e+10,Real Estate
8,336,CCI,6.114205e+10,Real Estate
9,1090,PSA,5.259268e+10,Real Estate


In [27]:
tickers = list(df.symbol)

In [11]:
def Optimize(tickers):
    # Filter ttfs
    post_filter = FilterTTF(tickers, verbatim=False)
    tickers = [ticker for ticker in post_filter ]


    # Run optimizer
    params = {'bounds': (0.01, 0.3), 'penalties': {'hs': 0.005, 'hi': 0.005, 'b': 0.05}}
    optimizer = GainyOptimizer(tickers, dt_today, benchmark='SPY', lookback=9)
    opt_res = optimizer.OptimizePortfolioRiskBudget(params=params)
    
    return opt_res

In [31]:
# Keep optimizing while there are still default weights
min_weight=0

i = 0
while min_weight<=0.01:
    i += 1
    print(i)
    tmp =  Optimize(tickers)
    min_weight = min([i for x,i in tmp.items()])
    tickers = [x for x,i in tmp.items() if i>0.01]
    print(tickers)

tmp

1
['COST', 'SYY', 'PLD', 'EQIX', 'WFC', 'BAC', 'WBA', 'NEE', 'JPM', 'COP', 'HDB', 'AON', 'AMT', 'MMC', 'MET', 'CCI', 'PSA', 'CVX', 'SHEL', 'XOM', 'SRE', 'RY', 'D', 'SO', 'CB', 'WMT']
2
['COST', 'SYY', 'PLD', 'EQIX', 'WFC', 'BAC', 'WBA', 'NEE', 'JPM', 'COP', 'HDB', 'AON', 'AMT', 'MMC', 'MET', 'CCI', 'PSA', 'CVX', 'SHEL', 'XOM', 'SRE', 'RY', 'SO']
3
['COST', 'SYY', 'PLD', 'EQIX', 'WFC', 'WBA', 'BAC', 'NEE', 'JPM', 'COP', 'HDB', 'AON', 'AMT', 'MMC', 'MET', 'CCI', 'PSA', 'CVX', 'SHEL', 'SRE', 'XOM', 'RY']
4
['COST', 'SYY', 'PLD', 'EQIX', 'WFC', 'WBA', 'NEE', 'BAC', 'JPM', 'COP', 'HDB', 'AON', 'AMT', 'MMC', 'MET', 'CCI', 'PSA', 'CVX', 'SHEL', 'SRE', 'XOM', 'RY']


{'COST': 0.06959876953771162,
 'SYY': 0.06569459969255963,
 'PLD': 0.057331022691310005,
 'EQIX': 0.057069734648924786,
 'WFC': 0.055585674935506424,
 'WBA': 0.054929298055136945,
 'NEE': 0.05423180424034637,
 'BAC': 0.05397929086006831,
 'JPM': 0.048512007778774784,
 'COP': 0.047406432202654024,
 'HDB': 0.04612274475093007,
 'AON': 0.045371880271237196,
 'AMT': 0.04516345024754449,
 'MMC': 0.04473190958967649,
 'MET': 0.04114222569773914,
 'CCI': 0.039352145977459146,
 'PSA': 0.035053902456872905,
 'CVX': 0.034124389201538145,
 'SHEL': 0.03250269380461106,
 'SRE': 0.027387644363954123,
 'XOM': 0.02497856266884426,
 'RY': 0.01972981632660012}

In [32]:
tmp = pd.DataFrame.from_dict(tmp, orient='index').reset_index()
tmp.columns=['symbol','weight']

df = tmp.merge(df, on='symbol')

In [33]:
df.groupby('gic_group').weight.sum()

gic_group
Banks                       0.223930
Energy                      0.139012
Food & Staples Retailing    0.190223
Insurance                   0.131246
Real Estate                 0.233970
Utilities                   0.081619
Name: weight, dtype: float64

In [34]:
df

,symbol,weight,index,market_capitalization,gic_group
0,COST,0.069599,1164,2.362000e+11,Food & Staples Retailing
1,SYY,0.065695,299,4.388643e+10,Food & Staples Retailing
2,PLD,0.057331,1103,1.074372e+11,Real Estate
3,EQIX,0.057070,525,6.243175e+10,Real Estate
4,WFC,0.055586,1115,1.807697e+11,Banks
5,WBA,0.054929,572,3.602811e+10,Food & Staples Retailing
6,NEE,0.054232,1084,1.695842e+11,Utilities
7,BAC,0.053979,1161,3.024456e+11,Banks
8,JPM,0.048512,878,4.010858e+11,Banks
9,COP,0.047406,142,1.579270e+11,Energy


# Update csv with Inflation Proof portfolio

In [35]:
all_ttfs = pd.read_csv("./all_ttf_weights_20221124.csv")
all_ttfs.loc[all_ttfs.ttf_id==275]

,symbol,weight,date,ttf_id,optimized_at
368,COST,0.101096,2022-11-24,275,2022-11-25 15:25:50.158376
369,BAC,0.080599,2022-11-24,275,2022-11-25 15:25:50.158376
370,PLD,0.079596,2022-11-24,275,2022-11-25 15:25:50.158376
371,JPM,0.074562,2022-11-24,275,2022-11-25 15:25:50.158376
372,NEE,0.072478,2022-11-24,275,2022-11-25 15:25:50.158376
373,AON,0.070764,2022-11-24,275,2022-11-25 15:25:50.158376
374,MMC,0.070366,2022-11-24,275,2022-11-25 15:25:50.158376
375,AMT,0.068000,2022-11-24,275,2022-11-25 15:25:50.158376
376,CCI,0.062447,2022-11-24,275,2022-11-25 15:25:50.158376
377,TTE,0.054097,2022-11-24,275,2022-11-25 15:25:50.158376


In [36]:
to_add = df[['symbol', 'weight']]
to_add['date']=all_ttfs.date.max()
to_add['ttf_id'] = 275
to_add['optimized_at'] = all_ttfs.loc[all_ttfs.ttf_id==275,'optimized_at'].iloc[0]
to_add

,symbol,weight,date,ttf_id,optimized_at
0,COST,0.069599,2022-11-24,275,2022-11-25 15:25:50.158376
1,SYY,0.065695,2022-11-24,275,2022-11-25 15:25:50.158376
2,PLD,0.057331,2022-11-24,275,2022-11-25 15:25:50.158376
3,EQIX,0.057070,2022-11-24,275,2022-11-25 15:25:50.158376
4,WFC,0.055586,2022-11-24,275,2022-11-25 15:25:50.158376
5,WBA,0.054929,2022-11-24,275,2022-11-25 15:25:50.158376
6,NEE,0.054232,2022-11-24,275,2022-11-25 15:25:50.158376
7,BAC,0.053979,2022-11-24,275,2022-11-25 15:25:50.158376
8,JPM,0.048512,2022-11-24,275,2022-11-25 15:25:50.158376
9,COP,0.047406,2022-11-24,275,2022-11-25 15:25:50.158376


In [37]:
all_ttfs = all_ttfs.loc[all_ttfs.ttf_id!=275]
all_ttfs = pd.concat([all_ttfs, to_add])
all_ttfs

,symbol,weight,date,ttf_id,optimized_at
0,TTC,0.119806,2022-11-24,225,2022-11-25 15:24:19.091462
1,PCAR,0.114910,2022-11-24,225,2022-11-25 15:24:19.091462
2,LNN,0.114193,2022-11-24,225,2022-11-25 15:24:19.091462
3,ALSN,0.109458,2022-11-24,225,2022-11-25 15:24:19.091462
4,OSK,0.105247,2022-11-24,225,2022-11-25 15:24:19.091462
...,...,...,...,...,...
17,CVX,0.034124,2022-11-24,275,2022-11-25 15:25:50.158376
18,SHEL,0.032503,2022-11-24,275,2022-11-25 15:25:50.158376
19,SRE,0.027388,2022-11-24,275,2022-11-25 15:25:50.158376
20,XOM,0.024979,2022-11-24,275,2022-11-25 15:25:50.158376


# Check

In [39]:
all_ttfs.groupby('ttf_id').weight.sum().min()

0.9999999999999989

In [40]:
all_ttfs.groupby('ttf_id').weight.sum().max()

1.0

In [42]:
len(all_ttfs.ttf_id.unique())

78